# 2. 온라인 선호지수 개발

## 2.2 협업 필터링을 통한 선호도 점수 행렬 도출

In [2]:
import numpy as np
import pandas as pd
import sklearn
from pandas import Series, DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import os

### 1) 구매 count를 이용한 온라인 선호지수 생성

In [99]:
#Purchase_Count : Developing Online Preference 2에서 생성한 client 대 상품군 구매 수랑 matrix 
#데이터 불러오기
Purchase_Count = pd.read_csv("C:/Users/lsh93/Desktop/data output/Purchase_Count.csv")
Purchase_Count = Purchase_Count.rename(columns = {'Unnamed: 0':'CLNT_ID'})
Purchase_Count.head()

,CLNT_ID,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
0,4140076,23.0,52.0,17.0,5.0,5.0,24.0,8.0,8.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4148106,3.0,3.0,0.0,0.0,0.0,9.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5298612,1.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5553084,7.0,13.0,21.0,0.0,3.0,10.0,11.0,19.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5639922,11.0,16.0,9.0,0.0,0.0,17.0,4.0,55.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0


In [97]:
#Customer Segmentation에서 샘플링해 온 client 목록 가져오기
op_clients = pd.read_csv("C:/Users/lsh93/Desktop/data output/op_clients.csv") 
len(op_clients)

8542

In [100]:
#Purchase_Count에서 뽑아오기
Purchase_Count = pd.merge(Purchase_Count, op_clients, how = 'inner', on ='CLNT_ID')
len(Purchase_Count['CLNT_ID'].unique())

6450

In [29]:
row_names = Purchase_Count['CLNT_ID'].as_matrix()
row_names

C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([4140076, 4148106, 5298612, ..., 3652718,  718983, 2909364],
      dtype=int64)

In [31]:
col_names = Purchase_Count.columns.values
col_names = col_names[1:]
col_names

array(['스포츠패션', '남성의류', '유아동의류', '청소/세탁/욕실용품', '완구', '패션잡화', '시즌스포츠',
       '여성의류', '인테리어/조명', '속옷/양말/홈웨어', '과일', '화장품/뷰티케어', '건강식품',
       '문구/사무용품', '주방잡화', '생활/주방가전', '침구/수예', '컴퓨터', '계절가전', '가구',
       '식기/조리기구', '모바일', '헬스/피트니스', '아웃도어/레저', '퍼스널케어', '세제/위생',
       '출산/육아용품', '음료', '구기/필드스포츠', '냉장/세탁가전', '냉동식품', '냉장식품', '원예/애완',
       '상품권', '자동차용품', '축산물', '영상/음향가전'], dtype=object)

In [33]:
PC = Purchase_Count.as_matrix()
PC

C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[4.140076e+06, 2.300000e+01, 5.200000e+01, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [4.148106e+06, 3.000000e+00, 3.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [5.298612e+06, 1.000000e+00, 4.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       ...,
       [3.652718e+06, 0.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [7.189830e+05, 0.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00],
       [2.909364e+06, 0.000000e+00, 0.000000e+00, ..., 0.000000e+00,
        0.000000e+00, 0.000000e+00]])

In [34]:
PC = PC[:,1:]

In [35]:
PC.shape

(6450, 37)

In [36]:
temp = pd.DataFrame(PC, columns = col_names, index = row_names)

In [37]:
#일부 row만 출력
temp[0:15] #0번째 row부터 14번째 row까지 출력됨 

,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,속옷/양말/홈웨어,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
4140076,23.0,52.0,17.0,5.0,5.0,24.0,8.0,8.0,7.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4148106,3.0,3.0,0.0,0.0,0.0,9.0,7.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5298612,1.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5553084,7.0,13.0,21.0,0.0,3.0,10.0,11.0,19.0,0.0,6.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5639922,11.0,16.0,9.0,0.0,0.0,17.0,4.0,55.0,0.0,12.0,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0
5650295,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5851368,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4128075,10.0,0.0,0.0,0.0,0.0,1.0,8.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4207528,3.0,2.0,1.0,2.0,0.0,7.0,0.0,5.0,2.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4219875,5.0,0.0,0.0,6.0,0.0,2.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
#이제 이 Purchase_Count를 이용할 것임
Purchase_Count = temp
print(Purchase_Count.head())
print(Purchase_Count.shape)

         스포츠패션  남성의류  유아동의류  청소/세탁/욕실용품   완구  패션잡화  시즌스포츠  여성의류  인테리어/조명  \
4140076   23.0  52.0   17.0         5.0  5.0  24.0    8.0   8.0      7.0   
4148106    3.0   3.0    0.0         0.0  0.0   9.0    7.0   0.0      0.0   
5298612    1.0   4.0    2.0         0.0  0.0   0.0    0.0   0.0      0.0   
5553084    7.0  13.0   21.0         0.0  3.0  10.0   11.0  19.0      0.0   
5639922   11.0  16.0    9.0         0.0  0.0  17.0    4.0  55.0      0.0   

         속옷/양말/홈웨어   ...      음료  구기/필드스포츠  냉장/세탁가전  냉동식품  냉장식품  원예/애완  상품권  \
4140076        4.0   ...     0.0       0.0      0.0   0.0   0.0    0.0  0.0   
4148106        0.0   ...     0.0       0.0      0.0   0.0   0.0    0.0  0.0   
5298612        0.0   ...     0.0       0.0      0.0   0.0   0.0    0.0  0.0   
5553084        6.0   ...     0.0       3.0      0.0   0.0   0.0    0.0  0.0   
5639922       12.0   ...     0.0       0.0      0.0   0.0   0.0   16.0  0.0   

         자동차용품  축산물  영상/음향가전  
4140076    0.0  0.0      0.0  
414810

In [39]:
#안에 있는 element값만 가져온 것
print(PC)
print(PC.shape)

[[23. 52. 17. ...  0.  0.  0.]
 [ 3.  3.  0. ...  0.  0.  0.]
 [ 1.  4.  2. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]
(6450, 37)


In [40]:
sparsity = float(len(PC.nonzero()[0]))
sparsity /= (PC.shape[0] * PC.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%',format(sparsity))

Sparsity: {:4.2f}% 14.197779174523362


In [41]:
from sklearn.model_selection import train_test_split

In [42]:
#test set과 training set으로 나누기
PC_train, PC_test = train_test_split(Purchase_Count, test_size = 0.33, random_state = 50)

In [43]:
#print(PC_train)
print(PC_train.shape)
#print(PC_test)
print(PC_test.shape)

(4321, 37)
(2129, 37)


In [44]:
PC_train = pd.DataFrame(PC_train)
PC_train.head()

,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,속옷/양말/홈웨어,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
5174505,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
502099,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6139346,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
853755,1.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1304095,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
PC_test = pd.DataFrame(PC_test)
PC_train.head()

,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,속옷/양말/홈웨어,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
5174505,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
502099,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6139346,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
853755,1.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1304095,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Cosine Distance를 이용한 선호점수 계산

In [36]:
#PC_train과 PC_test를 이용

,스포츠패션,남성의류,유아동의류,청소/세탁/욕실용품,완구,패션잡화,시즌스포츠,여성의류,인테리어/조명,속옷/양말/홈웨어,...,음료,구기/필드스포츠,냉장/세탁가전,냉동식품,냉장식품,원예/애완,상품권,자동차용품,축산물,영상/음향가전
598339,8.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1386382,4.0,1.0,5.0,0.0,0.0,1.0,7.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5853909,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5256290,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
#array 형태로 변환
train_mat = PC_train.as_matrix()
test_mat = PC_test.as_matrix()

C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [47]:
train_mat.shape

(4321, 37)

In [48]:
test_mat.shape

(2129, 37)

####  Cosine Similarity 계산

In [49]:
#코사인 유사도 계산 
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

In [50]:
dist_out = 1 - pairwise_distances(train_mat, metric="cosine")

In [51]:
dist_out 

array([[1.        , 0.58554004, 0.37796447, ..., 0.06900656, 0.1076244 ,
        0.        ],
       [0.58554004, 1.        , 0.77459667, ..., 0.42426407, 0.03676073,
        0.        ],
       [0.37796447, 0.77459667, 1.        , ..., 0.54772256, 0.0474579 ,
        0.        ],
       ...,
       [0.06900656, 0.42426407, 0.54772256, ..., 1.        , 0.10397505,
        0.        ],
       [0.1076244 , 0.03676073, 0.0474579 , ..., 0.10397505, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [52]:
dist_out.shape

(4321, 4321)

In [53]:
#Client의 알려지지 않은 평가 예측 
#거리행렬과 평가행렬의 내적을 구하고 평가수의 데이터를 정규화해 계산
user_pred = dist_out.dot(train_mat)/np.array([np.abs(dist_out).sum(axis = 1)]).T

In [54]:
from sklearn.metrics import mean_squared_error
def get_mse(pred,actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    
    return mean_squared_error(pred, actual)

In [55]:
#모델의 예측오차율
get_mse(user_pred, train_mat)

158.61687892338207

In [56]:
#테스트 데이터를 사용해 정확도를 확인해보기
get_mse(user_pred, test_mat)

274.7326421622518

In [57]:
from sklearn.neighbors import NearestNeighbors

In [58]:
k = 5
neigh = NearestNeighbors(k, 'cosine')

In [60]:
neigh.fit(train_mat)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2,
         radius='cosine')

In [61]:
#각 사용자별로 가장 유사한 5명의 사용자 찾고 유사도 계산
top_k_distances, top_k_users = neigh.kneighbors(train_mat, return_distance = True)

In [62]:
top_k_distances.shape

(4321, 5)

In [63]:
top_k_users.shape

(4321, 5)

In [64]:
top_k_users[4]

array([  4, 541, 463, 295, 416], dtype=int64)

In [65]:
#훈련 데이터의 알려지지 않은 구매 예측
user_pred_k = np.zeros(train_mat.shape)

In [66]:
for i in range(PC_train.shape[0]):
    user_pred_k[i,:] = top_k_distances[i].T.dot(train_mat[top_k_users][i]) / np.array([np.abs(top_k_distances[i].T).sum(axis = 0)]).T  

C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [67]:
user_pred_k.shape

(4321, 37)

In [68]:
user_pred_k

array([[0.25294283, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [1.51522887, 8.19397379, 5.26311942, ..., 0.        , 0.        ,
        0.        ],
       [0.29228571, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [69]:
#nan값 있어서 mse계산할 수 없음. 
#왜 nan값이 나올까,,
online_pref = pd.DataFrame(user_pred_k)
online_pref[0:10]

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,0.252943,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.505886,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.241172,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.768688,0.219835,0.711893,0.599168,0.0,1.729233,10.982043,0.000000,0.0,0.231312,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.445469,2.174760,0.000000,0.000000,0.0,0.564832,3.196442,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
6,0.000000,0.000000,0.500000,0.000000,0.0,1.500000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
7,3.253723,1.794683,6.267690,0.000000,0.0,2.536305,1.497210,3.790028,0.0,0.512102,...,0.0,0.253723,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
8,0.505421,2.225206,1.502075,0.000000,0.0,3.748963,0.767299,6.214365,0.0,2.255187,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
9,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,2.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


- nan가 들어간 row 전체를 제거한다. 대체하는 건 근거가 없으므로

In [70]:
#nan가 들어간 element의 row의 인덱스 반환
na_element_index = np.where(np.isnan(online_pref))
na_element_index

(array([   4,    4,    4, ..., 4318, 4318, 4318], dtype=int64),
 array([ 0,  1,  2, ..., 34, 35, 36], dtype=int64))

In [71]:
#nan가 들어간 row의 인덱스 반환
na_row_index = np.unique(na_element_index[0])
na_row_index

array([   4,   13,   19,   24,   31,   32,   33,   37,   38,   39,   42,
         46,   54,   57,   62,   64,   73,   76,   77,   87,  110,  129,
        134,  149,  153,  158,  160,  161,  165,  171,  172,  176,  186,
        201,  210,  222,  229,  233,  237,  239,  240,  250,  251,  259,
        262,  269,  273,  274,  295,  297,  309,  311,  312,  313,  321,
        334,  337,  339,  353,  358,  359,  363,  376,  377,  384,  390,
        394,  395,  416,  419,  442,  443,  444,  446,  448,  456,  458,
        463,  464,  472,  474,  480,  485,  495,  502,  503,  504,  508,
        516,  528,  534,  541,  542,  545,  561,  569,  577,  578,  582,
        599,  603,  604,  609,  615,  627,  629,  634,  644,  652,  663,
        680,  684,  692,  696,  698,  704,  713,  715,  727,  729,  739,
        754,  769,  770,  771,  785,  786,  787,  791,  800,  804,  809,
        819,  825,  828,  833,  834,  839,  845,  851,  853,  862,  865,
        875,  877,  879,  896,  898,  905,  912,  9

In [74]:
#제거할 client id의 목록
#temp_train 데이터프레임에서 반환
removed_CLNT_ID = PC_train.index.values[na_row_index]
removed_CLNT_ID.shape

(662,)

In [76]:
#제거하지 않고 남는 Client id의 목록
existed_CLNT_ID = np.delete(PC_train.index.values,na_row_index)
existed_CLNT_ID.shape

(3659,)

In [77]:
#이제 online_pref에서 nan가 들어간 row 제거
online_pref2 = online_pref.dropna()
online_pref2.shape

(3659, 37)

In [78]:
#결측치가 제거된 dataframe을 array로 바꾼다
user_pred_k2 = online_pref2.as_matrix()
user_pred_k2.shape

C:\Users\lsh93\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


(3659, 37)

In [79]:
#결측치 row제거된 선호지수행렬 
user_pred_k2

array([[0.25294283, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.51522887, 8.19397379, 5.26311942, ..., 0.        , 0.        ,
        0.        ],
       [0.29228571, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [94]:
na_row_index.shape

(662,)

In [110]:
#본래 training data에서도 nan이 나온 index를 제외해준다
train_mat2 = train_mat
exist_idx = np.delete([range(online_pref.shape[0])],na_row_index)
train_mat2 = train_mat2[exist_idx,:]
train_mat2.shape

(3659, 37)

In [105]:
#모델의 개선 여부 확인 -> 개선됨! 
get_mse(user_pred_k2, train_mat2)

75.7763597902311

In [106]:
get_mse(user_pred_k2, test_mat)

299.4609724630707

- 선호점수 matrix 생성 (index, column 모두 달아서)

In [107]:
user_pred_k2 

array([[0.25294283, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.51522887, 8.19397379, 5.26311942, ..., 0.        , 0.        ,
        0.        ],
       [0.29228571, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [86]:
#제거하지 않고 남은 index의 목록
existed_CLNT_ID

array([5174505,  502099, 6139346, ..., 5148951, 1844977, 3195953],
      dtype=int64)

In [87]:
col_names = Purchase_Count.columns.values
row_names = existed_CLNT_ID
print(col_names.shape)
print(row_names.shape)
print(user_pred_k2.shape)

(37,)
(3659,)
(3659, 37)


In [88]:
#사용할 수 있는 구매 count 기반 온라인 선호지수
Online_Preference_Count = pd.DataFrame(user_pred_k2, columns = col_names, index = row_names)

In [111]:
Online_Preference_Count.head()
Online_Preference_Count.shape

(3659, 37)

In [93]:
Existed_Client1 = pd.DataFrame(Online_Preference_Count.index.values, columns = ['CLNT_ID'])
len(Existed_Client1['CLNT_ID'].unique())

3659

In [91]:
#데이터 내보내기
Online_Preference_Count.to_csv("C:/Users/lsh93/Desktop/data/UBCF2/Online_Preference_Count.csv", encoding="UTF-8")
#Existed_Client1.to_csv("C:/Users/lsh93/Desktop/data/UBCF2/Existed_Client1.csv", encoding="UTF-8")